# Wordle Solver

In [1]:
# import functions
from wordle_helpers import *

In [2]:
# help(main)

In [3]:
# Main function that returns a list of possible words
green_letters = ''
green_positions = []
yellow_letters = 'ASY'
yellow_positions = [3,1,3,]
bad_letters = 'CRNETLO'

# Main function that returns a list of possible words
words = main(green_letters, green_positions, yellow_letters, yellow_positions, bad_letters)
print(words)

['assay', 'daisy', 'gassy']


In [4]:
# Repetitive letters
letters_df = repetitive_letters(wordle_list=words)
print(letters_df.to_string(index=False))

Letters  Count
      S      5
      A      4
      Y      3
      D      1
      I      1
      G      1


#### Choosing next word

In [5]:
# Main function that returns a list of possible words
green_letters = ''
green_positions = []
yellow_letters = 'LTYOS'
yellow_positions = []
bad_letters = ''

# Main function that returns a list of possible words
words = main(
    green_letters,
    green_positions,
    yellow_letters,
    yellow_positions,
    bad_letters,
    wordle_list_path="wordle-possible-words",
)
print(words)

['stylo', 'tosyl']


In [6]:
# help(score)

In [7]:
df = load_data(file_name='wordle_ranking.csv')
df

,Names,Games_Won
0,Murilo,211
1,Barbara,86
2,Draw,213


In [8]:
# Update score
score_df = score()
score_df

,Names,Games_Won
0,Murilo,212
1,Barbara,86
2,Draw,213


In [9]:
# Print results
print(score_df.to_string(index=False))

  Names  Games_Won
 Murilo        212
Barbara         86
   Draw        213


In [10]:
# help(reset_score)

#### Set custom score

In [11]:
# help(set_score)

In [12]:
# # Set custom score
# df = set_score(m_score=196, b_score=79, draw_score=204)
# df

#### Reset score

In [13]:
## Reset score
# reset_score()

In [14]:
# Churrasco amigo secreto
varejao = {
    'tulipa': [0.558, 32.59],
    'coxinha_asa': [0.538, 17.99],
    'linguica': [2.048, 24.99],
}

enxuto = {
    'picanha1': [1.410, 59.90],
    'picanha2': [1.516, 59.90],
}

covabra = {
    # 'carvao': [1, 59.99],
    'queijo_criou': [0.342, 96.90],
    # 'queijo_scala': [2, 29.99],
    'pao_alho': [2, 12.69],
    'cebola': [1.365, 9.90],
    'tomate': [1.130, 10.90]
}

covabra_desconto = 10.18 + 8.13

def calculate_total(items_dict:dict) -> float:
    total = sum(round(price[0] * price[1], 2) for _, price in items_dict.items())
    total = round(total, 2)
    return total

varejao_total = calculate_total(items_dict=varejao)
enxuto_total = calculate_total(items_dict=enxuto)
covabra_total = calculate_total(items_dict=covabra) - covabra_desconto
print("Total for each supermarket")
print(f"Varejao total: \tR${varejao_total}")
print(f"Enxuto total: \tR${enxuto_total}")
print(f"Covabra total: \tR${round(covabra_total, 2)}")

# Add everything
churrasco_total = varejao_total + enxuto_total + covabra_total
print(f"Churrasco total: R${churrasco_total}")
number_people = 13
price_each_person = round(churrasco_total / number_people, 2)
print(f"Price for each person: R${price_each_person}")

Total for each supermarket
Varejao total: 	R$79.05
Enxuto total: 	R$175.27
Covabra total: 	R$66.04
Churrasco total: R$320.36
Price for each person: R$24.64
